# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Load-data" data-toc-modified-id="Load-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load data</a></div><div class="lev1 toc-item"><a href="#Create-Timeseries-of-Daily-Flight-Counts" data-toc-modified-id="Create-Timeseries-of-Daily-Flight-Counts-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create Timeseries of Daily Flight Counts</a></div><div class="lev2 toc-item"><a href="#Create-interactive-plot-with-plotly/cufflinks" data-toc-modified-id="Create-interactive-plot-with-plotly/cufflinks-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Create interactive plot with plotly/cufflinks</a></div><div class="lev2 toc-item"><a href="#Flight-counts-tend-to-be-less-on-Saturdays" data-toc-modified-id="Flight-counts-tend-to-be-less-on-Saturdays-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Flight counts tend to be less on Saturdays</a></div><div class="lev1 toc-item"><a href="#Repeat-analysis-for-NY,-LA,-Houston." data-toc-modified-id="Repeat-analysis-for-NY,-LA,-Houston.-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Repeat analysis for NY, LA, Houston.</a></div><div class="lev2 toc-item"><a href="#Result-plots" data-toc-modified-id="Result-plots-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Result plots</a></div><div class="lev1 toc-item"><a href="#Which-state-had-the-most-air-traffics?" data-toc-modified-id="Which-state-had-the-most-air-traffics?-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Which state had the most air traffics?</a></div><div class="lev2 toc-item"><a href="#Choropleth-figure" data-toc-modified-id="Choropleth-figure-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Choropleth figure</a></div><div class="lev2 toc-item"><a href="#Choropleth-after-normalizing-over-state-population" data-toc-modified-id="Choropleth-after-normalizing-over-state-population-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Choropleth after normalizing over state population</a></div><div class="lev2 toc-item"><a href="#Weekday-patterns-for-each-state" data-toc-modified-id="Weekday-patterns-for-each-state-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Weekday patterns for each state</a></div><div class="lev1 toc-item"><a href="#Regress-out-effect-from-&quot;DAY-OF-WEEK&quot;" data-toc-modified-id="Regress-out-effect-from-&quot;DAY-OF-WEEK&quot;-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Regress out effect from "DAY OF WEEK"</a></div><div class="lev2 toc-item"><a href="#Plot-residuals-after-&quot;weekday&quot;-regression" data-toc-modified-id="Plot-residuals-after-&quot;weekday&quot;-regression-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Plot residuals after "weekday" regression</a></div><div class="lev2 toc-item"><a href="#Get-&quot;Lagged&quot;-residual" data-toc-modified-id="Get-&quot;Lagged&quot;-residual-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Get "Lagged" residual</a></div><div class="lev2 toc-item"><a href="#Create-subplots-with-all-3-plots" data-toc-modified-id="Create-subplots-with-all-3-plots-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Create subplots with all 3 plots</a></div><div class="lev2 toc-item"><a href="#Histograms" data-toc-modified-id="Histograms-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Histograms</a></div>

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn.apionly as sns
import plotly.plotly as py
import calendar

from datetime import datetime
from pprint import pprint
from IPython.display import display

import cufflinks as cf
cf.set_config_file(theme='ggplot')

import util

# limit output to avoid cluttering screen
pd.options.display.max_rows = 20

In [3]:
# name of output files to prepend with
outfile = 'flight_count_analysis_'

# Load data

Load the **airport data**, as well as the **lookup-table* I created [here](http://takwatanabe.me/airtraffic/create_lookup_table.html).


In [4]:
df_data = util.load_airport_data()

 ... load dataframe from 2015-11.zip 
 ... load dataframe from 2015-12.zip 
 ... load dataframe from 2016-01.zip 
 ... load dataframe from 2016-02.zip 
 ... load dataframe from 2016-03.zip 
 ... load dataframe from 2016-04.zip 
 ... load dataframe from 2016-05.zip 
 ... load dataframe from 2016-06.zip 
 ... load dataframe from 2016-07.zip 
 ... load dataframe from 2016-08.zip 
 ... load dataframe from 2016-09.zip 
 ... load dataframe from 2016-10.zip 


In [5]:
df_data.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID
0,2015,4,11,4,3,14570,13930
1,2015,4,11,5,4,13930,14057
2,2015,4,11,6,5,13930,14057
3,2015,4,11,7,6,13930,14057
4,2015,4,11,8,7,13930,14057


In [6]:
# make the "day_of_week" explicit
hash_dayofweek = {1:'Mon', 2:'Tue', 3:'Wed', 4:'Thu', 5:'Fri', 6:'Sat', 7:'Sun'}
df_data['DAY_OF_WEEK'] = df_data['DAY_OF_WEEK'].map(lambda key: hash_dayofweek[key])
df_data.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID
0,2015,4,11,4,Wed,14570,13930
1,2015,4,11,5,Thu,13930,14057
2,2015,4,11,6,Fri,13930,14057
3,2015,4,11,7,Sat,13930,14057
4,2015,4,11,8,Sun,13930,14057


In [7]:
df_lookup = pd.read_csv('df_lookup.csv') # lookup table for the AIRPORT_ID above
df_lookup.head()

,Code,Description,Airport,City,State,lat,lon,latlon,City_State
0,10135,"Allentown/Bethlehem/Easton, PA: Lehigh Valley ...",Lehigh Valley International,Allentown/Bethlehem/Easton,PA,40.651650,-75.434746,"(40.651650399999994, -75.434746099999984)",Allentown/Bethlehem/Easton (PA)
1,10136,"Abilene, TX: Abilene Regional",Abilene Regional,Abilene,TX,32.448736,-99.733144,"(32.448736400000001, -99.733143900000002)",Abilene (TX)
2,10140,"Albuquerque, NM: Albuquerque International Sun...",Albuquerque International Sunport,Albuquerque,NM,35.043333,-106.612909,"(35.0433333, -106.6129085)",Albuquerque (NM)
3,10141,"Aberdeen, SD: Aberdeen Regional",Aberdeen Regional,Aberdeen,SD,45.453458,-98.417726,"(45.453458300000001, -98.417726099999996)",Aberdeen (SD)
4,10146,"Albany, GA: Southwest Georgia Regional",Southwest Georgia Regional,Albany,GA,31.535671,-84.193905,"(31.535671100000002, -84.193904900000007)",Albany (GA)


In [8]:
# create hash-table to convert Airport "Code" to "City_State" 
# (combination of city/state is verified to be unique with the scope of this dataset)
hash_lookup = df_lookup.set_index('Code')['City_State'].to_dict()
pprint({k: hash_lookup[k] for k in hash_lookup.keys()[:10]})

# also create hash-table for airport names
hash_airport = df_lookup.set_index('Code')['Airport'].to_dict()
pprint({k: hash_airport[k] for k in hash_lookup.keys()[:10]})

{10245: 'King Salmon (AK)',
 10754: 'Barrow (AK)',
 11267: 'Dayton (OH)',
 11274: 'Dubuque (IA)',
 11278: 'Washington (DC) [R.Reagan]',
 11778: 'Fort Smith (AR)',
 13230: 'Harrisburg (PA)',
 13830: 'Kahului (HI)',
 14696: 'South Bend (IN)',
 15412: 'Knoxville (TN)'}
{10245: 'King Salmon Airport',
 10754: 'Wiley Post/Will Rogers Memorial',
 11267: 'James M Cox/Dayton International',
 11274: 'Dubuque Regional',
 11278: 'Ronald Reagan Washington National',
 11778: 'Fort Smith Regional',
 13230: 'Harrisburg International',
 13830: 'Kahului Airport',
 14696: 'South Bend International',
 15412: 'McGhee Tyson'}


# Create Timeseries of Daily Flight Counts

- Here, I would like to analyze the trend in the **total daily flights** in the United States.

- To this end, we'll first construct a [Pandas TimeSeries](http://pandas.pydata.org/pandas-docs/stable/timeseries.html) DataFrame containing the daily Flight-count information.

In [9]:
# create a column containing "YEAR-MONTH-DAY"
df_data['time'] = ( df_data['YEAR'].astype(str) + '-' 
                  + df_data['MONTH'].astype(str) + '-' 
                  + df_data['DAY_OF_MONTH'].astype(str))

df_data.head()

,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,time
0,2015,4,11,4,Wed,14570,13930,2015-11-4
1,2015,4,11,5,Thu,13930,14057,2015-11-5
2,2015,4,11,6,Fri,13930,14057,2015-11-6
3,2015,4,11,7,Sat,13930,14057,2015-11-7
4,2015,4,11,8,Sun,13930,14057,2015-11-8


In [10]:
# create time-series of airtraffic counts
ts_flightcounts = pd.DataFrame(df_data['time'].value_counts()).rename(columns={'time':'counts'})
ts_flightcounts.index = ts_flightcounts.index.to_datetime()
ts_flightcounts.sort_index(inplace=True) # need to sort by date
ts_flightcounts.head(8)

,counts
2015-11-01,15652
2015-11-02,16596
2015-11-03,15918
2015-11-04,16363
2015-11-05,16619
2015-11-06,16600
2015-11-07,12793
2015-11-08,15679


In [11]:
# explicitly add extra date-info as dataframe columns (to apply `groupby` later)
ts_flightcounts['day']= ts_flightcounts.index.day
ts_flightcounts['month']= ts_flightcounts.index.month
ts_flightcounts['day_of_week'] = ts_flightcounts.index.dayofweek

ts_flightcounts.head()

,counts,day,month,day_of_week
2015-11-01,15652,1,11,6
2015-11-02,16596,2,11,0
2015-11-03,15918,3,11,1
2015-11-04,16363,4,11,2
2015-11-05,16619,5,11,3


In [12]:
# `dayofweek` uses encoding Monday=0 ... Sunday=6...make this explicit
ts_flightcounts['day_of_week'] = ts_flightcounts['day_of_week'].map({0:'Mon',
                                                                     1:'Tue',
                                                                     2:'Wed',
                                                                     3:'Thu',
                                                                     4:'Fri',
                                                                     5:'Sat',
                                                                     6:'Sun'}).astype(str)

ts_flightcounts.head()

,counts,day,month,day_of_week
2015-11-01,15652,1,11,Sun
2015-11-02,16596,2,11,Mon
2015-11-03,15918,3,11,Tue
2015-11-04,16363,4,11,Wed
2015-11-05,16619,5,11,Thu


In [13]:
# create hover_text object for plotly
hover_text= (
    ts_flightcounts['month'].astype(str) 
    + '/' 
    + ts_flightcounts['day'].astype(str)
    + ' (' + ts_flightcounts['day_of_week'] + ')'
).tolist()
print hover_text[:5]

['11/1 (Sun)', '11/2 (Mon)', '11/3 (Tue)', '11/4 (Wed)', '11/5 (Thu)']


## Create interactive plot with plotly/cufflinks

- I am a huge fan of [plotly](http://plot.ly/python/)...brings the distance between the data and user closer together :)


In [14]:
# see https://plot.ly/pandas/line-charts/
plt_options = dict(text=hover_text,color='pink')
title = 'Daily Airflight Counts in the US between 11/1/2015 - 10/31/2016'
title+= '<br>(hover over plot for dates; left-click to zoom)'

ts_flightcounts.iplot(y='counts',
                      filename=outfile+'plot_flightcounts',
                      title=title,
                      **plt_options)

- From the above time-series plot, we can see that the trend in the Flight-counts looks to be obscured by the effect from the ``day_of_week``



## Flight counts tend to be less on Saturdays

- below I made a few barplots of flight-counts by weekdays
- cleary, Saturday tends to have smaller flight-counts
- while this was somewhat expected, it's always nice to have the data reaffirm your intuition


In [15]:
df_data['DAY_OF_WEEK'].value_counts().iplot(kind='bar',title='US Flight-Counts by Week-days (1 year period)')

In [16]:
df_counts_month = df_data.groupby(['MONTH','DAY_OF_WEEK',])['YEAR'].count().unstack()
df_counts_month = df_counts_month[['Sun','Mon','Tue','Wed','Thu','Fri','Sat']] # reorder columns

In [79]:
df_counts_month.index = df_counts_month.index.map(lambda num: calendar.month_abbr[num])
df_counts_month

TypeError: list indices must be integers, not str

- the trend of Saturday having the smallest airflights holds generally true for each month
- intereting exceptions at **January** and **July**...perhaps this is a common vacation period (so businessday trend is eliminated)?
- for example, maybe there tends to be more family trips since children is on school vacation

In [19]:
title='US Flight-Counts by Months by Week-days (11/1/2015 - 10/31/2016)'
df_counts_month.iplot(kind='bar',title=title,xTitle='Month',yTitle='Counts')

# Repeat analysis for NY, LA, Houston.

- Now I'm curious to see what the trend looks like in major cities.

- Let's repeat the above analysis for NY, LA, and Houston (selected these cities since these are cities with large population that are geographically far apart from each other)


In [20]:
cities = ['New York','Los Angeles','Houston']

# get AIRPORT_ID codes corresponding to the above three cities
df_lookup[ df_lookup['City'].isin(cities) ]

,Code,Description,Airport,City,State,lat,lon,latlon,City_State
95,11495,"Houston, TX: Ellington",Ellington,Houston,TX,29.760427,-95.369803,"(29.7604267, -95.369802799999988)",Houston (TX) [Ell]
142,12191,"Houston, TX: William P Hobby",William P Hobby,Houston,TX,29.652951,-95.276651,"(29.6529506, -95.276650700000005)",Houston (TX) [WP.Hobby]
150,12266,"Houston, TX: George Bush Intercontinental/Houston",George Bush Intercontinental/Houston,Houston,TX,29.990220,-95.336783,"(29.990219899999996, -95.336782700000001)",Houston (TX) [G.Bush]
164,12478,"New York, NY: John F. Kennedy International",John F. Kennedy International,New York,NY,40.641311,-73.778139,"(40.641311100000003, -73.77813909999999)",New York (NY) [JFK]
174,12892,"Los Angeles, CA: Los Angeles International",Los Angeles International,Los Angeles,CA,33.941589,-118.408530,"(33.941588899999999, -118.40853)",Los Angeles (CA)
180,12953,"New York, NY: LaGuardia",LaGuardia,New York,NY,40.776927,-73.873966,"(40.776927100000002, -73.873965900000016)",New York (NY) [Lag]


- Well, both Houston and NY have multiple major airport.

- For the sake of simplicity of our analysis, we'll pool the flight counts from these airports.

In [64]:
cities = ['New York','Los Angeles','Houston']
city_codes = {'New York':[12478, 12953],
              'Los Angeles' : [12892],
               'Houston' : [11495,12191,12266]}

def filter_by_codelist(df_data,code_list):
    mask1 = df_data['ORIGIN_AIRPORT_ID'].isin(code_list)
    mask2 = df_data['DEST_AIRPORT_ID'].isin(code_list)
    return df_data[mask1 | mask2]

df_data_city = {city:[] for city in cities}

for city in cities:
    print city
    city_code = city_codes[city]
    df_data_city[city] = filter_by_codelist(df_data, city_code)
    
    # sanity check
    display(df_data_city[city].sample(3))
    


New York


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,time
457403,2016,1,3,27,Sun,12953,13232,2016-3-27
397311,2016,2,5,10,Tue,12953,11259,2016-5-10
106627,2016,1,3,23,Wed,12478,12892,2016-3-23


Los Angeles


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,time
465803,2016,3,7,23,Sat,14107,12892,2016-7-23
340102,2016,3,7,25,Mon,10821,12892,2016-7-25
353543,2016,4,10,16,Sun,12892,12264,2016-10-16


Houston


,YEAR,QUARTER,MONTH,DAY_OF_MONTH,DAY_OF_WEEK,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,time
295249,2016,3,9,29,Thu,12266,14679,2016-9-29
237403,2016,2,5,4,Wed,12266,14771,2016-5-4
327047,2016,4,10,16,Sun,12266,13495,2016-10-16


## Result plots

In [22]:
flight_counts = []
for city in cities:
    flight_counts.append(df_data_city[city]['DAY_OF_WEEK'].value_counts())
    
flight_counts = pd.DataFrame(flight_counts,index=cities)[['Sun','Mon','Tue','Wed','Thu','Fri','Sat']] # reorder columns
flight_counts

,Sun,Mon,Tue,Wed,Thu,Fri,Sat
New York,54774,59836,57450,58018,58680,58437,43946
Los Angeles,60730,63463,60909,61503,62119,62430,51961
Houston,55733,59645,55493,57025,58124,58234,45377


In [23]:

flight_counts.iplot(kind='bar')

In [24]:
figs = {city:[] for city in cities}
for city in cities:
    figs[city] = df_data_city[city].\
        groupby(['MONTH','DAY_OF_WEEK',])['YEAR'].\
        count().unstack()[['Sun','Mon','Tue','Wed','Thu','Fri','Sat']].\
        iplot(kind='bar',title=city, xTitle='Month',yTitle='Counts',asFigure=True)


In [25]:
py.iplot(figs['New York'],title='New York')

In [26]:
py.iplot(figs['Houston'],title='Houston')

In [27]:
py.iplot(figs['Los Angeles'],title='Los Angeles')

# Which state had the most air traffics?


In [83]:
states = sorted(list(df_lookup['State'].unique()))
print states[:8]

# initialize dict
state_codes = {state:[] for state in states}
for state in states:
    # get AIRPORT_ID codes corresponding to this state
    state_codes[state] = df_lookup[ df_lookup['State'] == state]['Code'].tolist()

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC']


In [84]:
pprint({k:state_codes[k] for k in state_codes.keys()[:5]})

{'DC': [11278, 12264],
 'VA': [10990, 13931, 14098, 14524, 14574],
 'WA': [10666, 11884, 14252, 14747],
 'WI': [10408, 11203, 11471, 11977, 13076, 13342, 13485, 14520],
 'WV': [11146]}


In [63]:
#|below just for sanity check
# for state in states[:5]:
#     print state
#     display(df_lookup[ df_lookup['Code'].isin(state_codes[state])] )

In [89]:
df_data_states = {state:[] for state in states}

for state in states:
    df_data_states[state] = filter_by_codelist(df_data, state_codes[state])

In [93]:
flight_counts_state_weekday = []
for state in states:
    flight_counts_state_weekday.append(df_data_states[state]['DAY_OF_WEEK'].value_counts())
    
flight_counts_state_weekday = pd.DataFrame(flight_counts_state,index=states)[['Sun','Mon','Tue','Wed','Thu','Fri','Sat']] # reorder columns
flight_counts_state_weekday

,Sun,Mon,Tue,Wed,Thu,Fri,Sat
AK,7578,7775,7415,7350,7630,7677,7393
AL,7154,7917,7565,7674,7734,7766,5315
AR,4746,5465,5309,5369,5350,5337,3560
AZ,50097,51292,48255,50236,50543,50824,45005
CA,174652,183544,175707,177915,179338,180126,147463
CO,69073,71859,68468,69778,70153,70366,59675
CT,5581,6061,5831,5855,5902,5928,5036
DC,32205,34385,33748,33941,33597,33554,24701
FL,126202,127999,123016,123941,125002,125420,124652
GA,110343,117193,114210,114653,114609,114734,90989


In [104]:
flight_counts_state = flight_counts_state_weekday.sum(axis=1).\
                          sort_values(ascending=False).to_frame(name='counts')
flight_counts_state

,counts
CA,1218745
TX,1041128
FL,876232
GA,776731
IL,702984
NY,495827
CO,479372
AZ,346252
NV,323313
NC,316300


In [105]:
flight_counts_state.iplot(kind='bar')

## Choropleth figure


In [126]:
flight_counts_state.index

Index([u'CA', u'TX', u'FL', u'GA', u'IL', u'NY', u'CO', u'AZ', u'NV', u'NC',
       u'MI', u'WA', u'MN', u'MA', u'NJ', u'DC', u'UT', u'PA', u'MO', u'MD',
       u'OH', u'TN', u'HI', u'OR', u'LA', u'WI', u'IN', u'VA', u'SC', u'OK',
       u'PR', u'AK', u'AL', u'NM', u'NE', u'ID', u'CT', u'AR', u'MT', u'KY',
       u'ND', u'IA', u'RI', u'MS', u'KS', u'WY', u'SD', u'ME', u'NH', u'VI',
       u'VT', u'WV', u'TT'],
      dtype='object')

In [231]:
flight_counts_state

# define data object
data = [dict(
        type='choropleth',
        autocolorscale = True,
        locations = flight_counts_state.index.tolist(),
        z = flight_counts_state.values,
        locationmode = 'USA-states',
        marker = dict(line = dict (color = 'rgb(0,0,0)',width = 2) ),
        colorbar = dict(title = "flight-counts")
)]

# define layout object
geo = dict(scope='usa',
           projection=dict( type='albers usa' ),
           showlakes = True,
           lakecolor = 'rgb(255, 255, 255)'
          )
layout = dict(geo=geo,
              margin = dict(b=0,l=0,r=0,t=40),
              title = 'Flight counts over last year (hover over for number)')

fig = dict( data=data, layout=layout )
py.iplot( fig )

## Choropleth after normalizing over state population

In [150]:
# 2015 estimate of state population saved va script ``get_us_state_populations.py``
df_state_popu = pd.read_csv('df_state_populations.csv')
df_state_popu.head()

,state,population
0,CA,39250017
1,TX,27862596
2,FL,20612439
3,NY,19745289
4,IL,12801539


In [137]:
df_flight_counts_state = flight_counts_state.reset_index().rename(columns={'index':'state'})
df_flight_counts_state.head()

,state,counts
0,CA,1218745
1,TX,1041128
2,FL,876232
3,GA,776731
4,IL,702984


In [152]:
df_flight_counts_state = df_flight_counts_state.merge(df_state_popu,on='state',how='left')
df_flight_counts_state.head()

,state,counts,population_x,population_y,population
0,CA,1218745,"39,250,017",39250017.0,39250017.0
1,TX,1041128,"27,862,596",27862596.0,27862596.0
2,FL,876232,"20,612,439",20612439.0,20612439.0
3,GA,776731,"10,310,371",10310371.0,10310371.0
4,IL,702984,"12,801,539",12801539.0,12801539.0


In [180]:
# normalize by state population
df_flight_counts_state['counts_n'] = df_flight_counts_state['counts']/df_flight_counts_state['population']
# df_flight_counts_state.head()
df_flight_counts_state.sort_values('counts_n',ascending=False)

,state,counts,population_x,population_y,population,counts_n
15,DC,226131,"681,170",681170.0,681170.0,0.331974
8,NV,323313,"2,940,058",2940058.0,2940058.0,0.109968
22,HI,139771,"1,428,557",1428557.0,1428557.0,0.097841
6,CO,479372,"5,540,545",5540545.0,5540545.0,0.086521
3,GA,776731,"10,310,371",10310371.0,10310371.0,0.075335
16,UT,218147,"3,051,217",3051217.0,3051217.0,0.071495
31,AK,52818,"741,894",741894.0,741894.0,0.071193
4,IL,702984,"12,801,539",12801539.0,12801539.0,0.054914
7,AZ,346252,"6,931,071",6931071.0,6931071.0,0.049956
12,MN,261571,"5,519,952",5519952.0,5519952.0,0.047386


In [214]:
df_flight_counts_state.query('state == "OK"')

,state,counts,population_x,population_y,population,counts_n
29,OK,62857,"3,923,561",3923561.0,3923561.0,0.01602


In [217]:
df_flight_counts_state_ = df_flight_counts_state.copy()
df_flight_counts_state_ = df_flight_counts_state_[ df_flight_counts_state_['state'] != 'DC' ].reset_index()

In [218]:
locations = df_flight_counts_state['state'].tolist()
locations

['CA',
 'TX',
 'FL',
 'GA',
 'IL',
 'NY',
 'CO',
 'AZ',
 'NV',
 'NC',
 'MI',
 'WA',
 'MN',
 'MA',
 'NJ',
 'DC',
 'UT',
 'PA',
 'MO',
 'MD',
 'OH',
 'TN',
 'HI',
 'OR',
 'LA',
 'WI',
 'IN',
 'VA',
 'SC',
 'OK',
 'PR',
 'AK',
 'AL',
 'NM',
 'NE',
 'ID',
 'CT',
 'AR',
 'MT',
 'KY',
 'ND',
 'IA',
 'RI',
 'MS',
 'KS',
 'WY',
 'SD',
 'ME',
 'NH',
 'VI',
 'VT',
 'WV',
 'TT']

In [230]:
'OK' in df_flight_counts_state['state'].tolist()
df_flight_counts_state_[df_flight_counts_state_['state']=='OK']

,index,state,counts,population_x,population_y,population,counts_n
28,29,OK,62857,"3,923,561",3923561.0,3923561.0,0.01602


In [219]:
# define data object
scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]
data = [dict(
        type='choropleth',
        #autocolorscale = False,
        #colorscale = scl,
        locations = df_flight_counts_state['state'].tolist(),
        z = df_flight_counts_state_['counts_n'].values,
        locationmode = 'USA-states',
        #marker = dict(line = dict (color = 'rgb(0,0,0)',width = 2) ),
        colorbar = dict(title = "flight-counts")
)]

# define layout object
geo = dict(scope='usa',
           projection=dict( type='albers usa' ),
           showlakes = True,
           lakecolor = 'rgb(255, 255, 255)'
          )
layout = dict(geo=geo,
              margin = dict(b=0,l=0,r=0,t=40),
              title = 'Flight counts over last year normalized over state population(hover over for number)')

fig = dict( data=data, layout=layout )
py.iplot( fig )

## Weekday patterns for each state

In [118]:
state_ranking = flight_counts_state.index.tolist()
print state_ranking

display(flight_counts_state_weekday.head())
display(flight_counts_state_weekday.reindex(state_ranking))

['CA', 'TX', 'FL', 'GA', 'IL', 'NY', 'CO', 'AZ', 'NV', 'NC', 'MI', 'WA', 'MN', 'MA', 'NJ', 'DC', 'UT', 'PA', 'MO', 'MD', 'OH', 'TN', 'HI', 'OR', 'LA', 'WI', 'IN', 'VA', 'SC', 'OK', 'PR', 'AK', 'AL', 'NM', 'NE', 'ID', 'CT', 'AR', 'MT', 'KY', 'ND', 'IA', 'RI', 'MS', 'KS', 'WY', 'SD', 'ME', 'NH', 'VI', 'VT', 'WV', 'TT']


,Sun,Mon,Tue,Wed,Thu,Fri,Sat
AK,7578,7775,7415,7350,7630,7677,7393
AL,7154,7917,7565,7674,7734,7766,5315
AR,4746,5465,5309,5369,5350,5337,3560
AZ,50097,51292,48255,50236,50543,50824,45005
CA,174652,183544,175707,177915,179338,180126,147463


,Sun,Mon,Tue,Wed,Thu,Fri,Sat
CA,174652,183544,175707,177915,179338,180126,147463
TX,149163,157490,150335,152656,153731,154344,123409
FL,126202,127999,123016,123941,125002,125420,124652
GA,110343,117193,114210,114653,114609,114734,90989
IL,100107,105914,103718,103898,103038,102957,83352
NY,69214,75567,72684,73430,74030,73798,57104
CO,69073,71859,68468,69778,70153,70366,59675
AZ,50097,51292,48255,50236,50543,50824,45005
NV,47092,47588,46013,46426,46954,47305,41935
NC,44027,47311,45908,47380,47004,47311,37359


In [124]:
flight_counts_state_weekday_sorted = flight_counts_state_weekday.reindex(state_ranking)
flight_counts_state_weekday_sorted.iplot(kind='bar',title='Click to zoom!')

In [120]:
flight_counts_state_weekday_sorted.ix[:15].iplot(kind='bar')

In [121]:
flight_counts_state_weekday_sorted.ix[15:30].iplot(kind='bar')

In [122]:
flight_counts_state_weekday_sorted.ix[30:45].iplot(kind='bar')

In [123]:
flight_counts_state_weekday_sorted.ix[45:].iplot(kind='bar')

# Regress out effect from "DAY OF WEEK"

- Apply linear regression using the "Day of the week" as the regressor.

- By studying the residual from this regression, we hope to see more salient pattern once the dominant effect of "day-of-week" is removed

In [28]:
import statsmodels.formula.api as smf
mod = smf.ols(formula = 'counts ~ C(day_of_week) - 1',data=ts_flightcounts).fit()

mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 counts   R-squared:                       0.553
Model:                            OLS   Adj. R-squared:                  0.545
Method:                 Least Squares   F-statistic:                     73.96
Date:                Tue, 10 Jan 2017   Prob (F-statistic):           9.34e-60
Time:                        16:26:24   Log-Likelihood:                -3004.6
No. Observations:                 366   AIC:                             6023.
Df Residuals:                     359   BIC:                             6051.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------
C(day_of_week)[Fri]  1.605e+04    124.514    128.866      0.000      1.58e+04  1.63e+04
C(day_of_week)[Mon]  1.604e+04    123.334    130.061      0.000      1.58e+04  1.63e+04
C(day_of_week)[Sat]  1.312e+04    124.514    105.345      0.000      1.29e+04  1.34e+04
C(day_of_week)[Sun]  1.518e+04    123.334    123.110      0.000      1.49e+04  1.54e+04
C(day_of_week)[Thu]  1.599e+04    124.514    128.415      0.000      1.57e+04  1.62e+04
C(day_of_week)[Tue]  1.578e+04    124.514    126.740      0.000      1.55e+04   1.6e+04
C(day_of_week)[Wed]  1.595e+04    124.514    128.122      0.000      1.57e+04  1.62e+04
==============================================================================
Omnibus:                      138.226   Durbin-Watson:                   0.904
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              802.535
Skew:                          -1.478   Prob(JB):                    5.39e-175
Kurtosis:                       9.625   Cond. No.                         1.01
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Plot residuals after "weekday" regression

In [29]:
# add residual information
ts_flightcounts['residual'] = mod.resid

title = 'Residual Airflight Counts in the US <br>(`day_of_week` as regressors)'
ts_flightcounts.iplot(y=['residual'],
                      filename=outfile+'plot_resid',
                      text=hover_text,
                      title=title)

## Get "Lagged" residual

- let's take this a step further, and compute and plot the "lagged" residual plot

- this is given by: ``resid_lag[t] = resid[t+1] - resid[t]``

In [30]:
# also add "lagged" residual information
ts_flightcounts['resid_lag'] = \
    ts_flightcounts['residual'].shift(1) - ts_flightcounts['residual']
    
title = 'Lagged Residual Plot of Airflight Counts in the US (left click to zoom)'
title+= '<br>("day-of-week" used as regressors; shaded region = +/-1.5 std-dev)'

annotations = {
    datetime(2015,11,26):'Thanksgiving',
    datetime(2015,12,24):'Christmas Eve',
    datetime(2015,12,31):'New Years',
    datetime(2016, 2, 7):'??? Something happen ???',
    datetime(2016, 5,29):'Memorials Day',
    datetime(2016, 7, 3):'Independence Day',
    datetime(2016, 9, 4):'Labor Day',
}

std_ = ts_flightcounts['resid_lag'].std()

In [31]:
ts_flightcounts['resid_lag'].iplot(
    filename=outfile+'plot_resid_lag',
    annotations=annotations,
    #hspan=[(-1.5*std_,1.5*std_)],
    hspan = dict(y0=-1.5*std_,y1=1.5*std_,opacity=0.2,color='teal',fill=True),
    text=hover_text,
    title=title)

## Create subplots with all 3 plots

In [32]:
title = 'Flight counts'
ts_flightcounts.iplot(y=['counts','residual','resid_lag'],
                      subplots=True, shape=(3,1),
                      text=hover_text,
                      shared_xaxes=True, 
                      title=title,
                      filename=outfile+'flightcounts_subplot')

## Histograms

In [33]:
#| meh, boring plot
# ts_flightcounts[['counts','residual','resid_lag']].iplot(kind='histogram',
#                       filename=outfile+'histogram',
#                       title='')

In [34]:
# ts_flightcounts[groups]

In [35]:
from plotly.tools import FigureFactory as FF

columns = ['counts','residual','resid_lag']
group_data = map(lambda col: ts_flightcounts[col].dropna().values,columns)
fig = FF.create_distplot(group_data,
                         group_labels=columns,
                         bin_size= 300,
                         curve_type='normal',#'kde' or 'normal'
)

py.iplot(fig, filename=outfile+'histogram2',title='Distributions among the 3 quantities of interest')